In [1]:
#导入模型参数
import argparse
import torch
import torch.optim as optim
from tqdm import tqdm
import numpy as np
import time
import os
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.optim.lr_scheduler import CosineAnnealingLR
import warnings

warnings.filterwarnings("ignore", message="not removing hydrogen atom without neighbors")
import sys
sys.path.append("/home/dwj/WWW/DDIsubgraph/pretrain")
from dataset.dataset import MoleculeDatasetWrapper
from models.model import Model001
from loss_utils.nt_xent import NTXentLoss
from loss_utils.weighted_nt_xent import Weighted_NTXentLoss
from loss_utils.motif_loss import Motif_Loss
parser = argparse.ArgumentParser(description='PyTorch implementation of pre-training of graph neural networks')
parser.add_argument('--device', type=int, default=1,
                    help='which gpu to use if any (default: 0)')
parser.add_argument('--batch_size', type=int, default=128,
                    help=' inputbatch size for training (default: 1024)')
parser.add_argument('--dataset', type=str, default='/home/dwj/WWW/DDIsubgraph/pretrain/data/Compsol.csv',
                    help='root directory of dataset.')

parser.add_argument('--valid_size', type=float, default=0.05,
                    help='valid_size (default: 0.2)')
parser.add_argument('--num_workers', type=int, default=0,
                    help=' the number of workers to load data (default: 8)')


parser.add_argument('--num_layer', type=int, default=4,
                    help='number of GNN message passing layers (default: 5).')
parser.add_argument('--emb_dim', type=int, default=300,
                    help='embedding dimensions (default: 300)')
parser.add_argument('--feat_dim', type=int, default=300,
                    help='embedding dimensions (default: 256)')
parser.add_argument('--dropout_gin', type=float, default=0,
                    help='dropout ratio (default: 0.2)')
parser.add_argument('--graph_pooling', type=str, default="mean",
                    help='graph level pooling (sum, mean, max, set2set, attention)')
parser.add_argument('--N', type=int, default=2,
                    help='num layer of transformer encoder')
parser.add_argument('--d_model', type=int, default=256,
                    help='embedding dimensions (default: 256)')
parser.add_argument('--d_ff', type=int, default=1024,
                    help='embedding dimensions (default: 1024)')
parser.add_argument('--h', type=int, default=8,
                    help='heads of transformer encoder(default: 8)')
parser.add_argument('--dropout_encoder', type=float, default=0.1,
                    help='dropout ratio (default: 0.1)')
parser.add_argument('--weight', type=int, default=1,
                    help='weight or not')

parser.add_argument('--epochs', type=int, default=10,
                    help='number of epochs to train (default: 100)')

parser.add_argument('--decay', type=float, default=0.00001,
                    help='weight decay (default: 0)')
parser.add_argument('--lr', type=float, default=0.0005,
                    help='learning rate (default: 0.001)')


parser.add_argument('--output_model_file', type=str, default='/home/dwj/WWW/DDIsubgraph/pretrain/save_model/pretrain/motif_loss+weight',
                    help='filename to output the pre-trained model')
parser.add_argument('--gama', type=float, default=0.2, help='weight of motif_loss')
args = parser.parse_args([])

torch.manual_seed(42)
np.random.seed(42)
device = torch.device("cuda:" + str(args.device)) if torch.cuda.is_available() else torch.device("cpu")
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
dataset = MoleculeDatasetWrapper(args.batch_size, args.num_workers, args.valid_size, args.dataset)   # dataset：一个txt文件路径，里面村的是smiles
train_loader, valid_loader = dataset.get_data_loaders()  # 导入数据集，编码，在dataset里
model = Model001(num_layer=args.num_layer, emb_dim=args.emb_dim, feat_dim=args.feat_dim,
                 dropout_gin=args.dropout_gin, pool=args.graph_pooling, device=device,
                 N=args.N, d_model=args.d_model, d_ff=args.d_ff, h=args.h,
                 dropout_encoder=args.dropout_encoder).to(device)   

3371 177


In [2]:
from datetime import datetime
import random
import pandas as pd
import numpy as np
import torch
from torch import nn 
from torch import optim
from torch.utils.data import DataLoader
from sklearn import metrics

import layers1
import models1
import custom_loss
import time
import torch.nn as nn
#'C','N','O', 'S','F','Si','P', 'Cl','Br','Mg','Na','Ca','Fe','As','Al','I','B','V','K','Tl',
#            'Yb','Sb','Sn','Ag','Pd','Co','Se','Ti','Zn','H', 'Li','Ge','Cu','Au','Ni','Cd','In',
#            'Mn','Zr','Cr','Pt','Hg','Pb','Unknown'

In [3]:
# Hyperparameters
n_atom_feats = 55
n_atom_hid = 256
rel_total = 86
lr = 1e-3
weight_decay = 5e-4
n_epochs = 300
neg_samples = 1
batch_size = 1024
data_size_ratio = 1
kge_dim = 384

In [4]:
def rmse_loss(pred, target):
    labels_tensor = torch.tensor([float(x) for x in target], dtype=torch.float32).unsqueeze(1).to(pred.device)
    return F.mse_loss(pred, labels_tensor )
def mae_loss(pred, target):
    # 将目标值转换为浮点数张量，并调整形状以匹配预测值
    labels_tensor = torch.tensor([float(x) for x in target], dtype=torch.float32).unsqueeze(1).to(pred.device)
    # 计算绝对误差
    absolute_errors = torch.abs(pred - labels_tensor)
    # 返回平均绝对误差
    return torch.mean(absolute_errors)

In [5]:
def train(model, train_data_loader, val_data_loader, loss_fn,  optimizer, n_epochs, device, scheduler=None):
    print('Starting training at', datetime.today())
    for i in range(1, n_epochs+1):
        start = time.time()
        train_loss = 0
        train_loss_pos = 0
        train_loss_neg = 0
        val_loss = 0
        val_loss_pos = 0
        val_loss_neg = 0

        sum1=0
        for batch in train_data_loader:
            model.train()
            p_score = model(batch,device)
            loss= rmse_loss(p_score,batch[2])
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            sum1+=len(batch[2]) 
            train_mae = mae_loss(p_score,batch[2])
            train_mae += train_mae.item() * len(p_score)
            train_rmse = rmse_loss(p_score,batch[2])
            train_rmse += train_rmse.item() * len(p_score)
        train_mae /= sum1
        train_rmse /= sum1

        with torch.no_grad():
            sum2=0
            for batch in val_data_loader:
                model.eval()
                p_score = model(batch,device)
                loss= rmse_loss(p_score,batch[2]) 
                #print(len(batch[2])  ) 
                 
                sum2+=len(batch[2])
                val_mae = mae_loss(p_score,batch[2])       
                val_mae += val_mae.item() * len(p_score)
                val_rmse = rmse_loss(p_score,batch[2])       
                val_rmse += val_rmse.item() * len(p_score)
            val_mae /= sum2
            val_rmse /= sum2
               
        if scheduler:
            scheduler.step()
        print(f"epoch {i} train_mae:  {train_mae} and val_mae:  {val_mae}  train_rmse:  {train_rmse} and val_rmse:  {val_rmse}/n")
        

In [6]:

loss = custom_loss.SigmoidLoss()
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.LambdaLR(optimizer, lambda epoch: 0.96 ** (epoch))

In [7]:
train(model, train_loader, valid_loader, loss, optimizer, n_epochs, device, scheduler)

Starting training at 2024-09-05 19:44:16.289914
epoch 1 train_mae:  0.05292055010795593 and val_mae:  1.7791236639022827  train_rmse:  0.1321474015712738 and val_rmse:  4.883437156677246/n
epoch 2 train_mae:  0.04661503806710243 and val_mae:  1.6452324390411377  train_rmse:  0.10689503699541092 and val_rmse:  4.876831531524658/n
epoch 3 train_mae:  0.057056035846471786 and val_mae:  1.82615327835083  train_rmse:  0.1343628615140915 and val_rmse:  4.957463264465332/n
epoch 4 train_mae:  0.05591604858636856 and val_mae:  1.6994951963424683  train_rmse:  0.13764172792434692 and val_rmse:  4.718388557434082/n
epoch 5 train_mae:  0.046449124813079834 and val_mae:  1.6967103481292725  train_rmse:  0.11426348239183426 and val_rmse:  4.670897006988525/n
epoch 6 train_mae:  0.05461429804563522 and val_mae:  1.7735737562179565  train_rmse:  0.12629222869873047 and val_rmse:  4.775106906890869/n
epoch 7 train_mae:  0.04656542092561722 and val_mae:  1.751615285873413  train_rmse:  0.10241900384426